In [ ]:
import pygame
import sys
import pandas as pd
import chess.pgn
import chess
import time
import numpy as np
from DeepChessEngine import DeepChessEngine

In [ ]:
DCE = DeepChessEngine(depth = 3)

In [ ]:
x = chess.Board()
y = chess.Move.from_uci("g1f3")
x.push(y)
print(type(x))
vg=np.array(x)
print(vg)

In [ ]:
def board_to_numpy(board):
    numpy_board = [
    ["", "", "", "", "", "", "", ""],
    ["", "", "", "", "", "", "", ""],
    ["", "", "", "", "", "", "", ""],
    ["", "", "", "", "", "", "", ""],
    ["", "", "", "", "", "", "", ""],
    ["", "", "", "", "", "", "", ""],
    ["", "", "", "", "", "", "", ""],
    ["", "", "", "", "", "", "", ""]
]

    piece_to_int = {
        chess.PAWN: "p",
        chess.KNIGHT: "n",
        chess.BISHOP: 'b',
        chess.ROOK: 'r',
        chess.KING: 'k',
        chess.QUEEN: 'q'
    }


    for square in chess.SQUARES:
        piece = board.piece_at(square)
        if piece is not None:
        
            piece_color = piece.color
            print(piece_color)
            piece_type = piece.piece_type

     
            if piece_color:
                piece_code = piece_to_int[piece_type].upper()
            else:
                piece_code = piece_to_int[piece_type]
            
            rank, file = chess.square_rank(square), chess.square_file(square)
            numpy_board[rank][file] = piece_code

    return numpy_board


In [ ]:
def get_position(coord):
    for i in range(8):
        if coord[0]//100 == i:
            break
    for j in range(8):
        if coord[1]//100 == j:
            break
    d = {0: 'a', 1: 'b', 2: 'c', 3: 'd', 4: 'e', 5: 'f', 6: 'g', 7: 'h'}
    return d[i] + str(j + 1)

In [ ]:
def selectSide():
    x = input("Write the side you want(w for white and b for black)")
    x = x.lower()
    if x=='w':
        return True
    else:
        return False

In [ ]:
board = chess.Board()

WIDTH, HEIGHT = 800, 800
ROWS, COLS = 8, 8
SQUARE_SIZE = WIDTH // COLS
clock=pygame.time.Clock()

WHITE = (240, 217, 181)
BLACK = (181, 136, 99)
GRAY = (128, 128, 128)


pygame.init()
window = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("Chess")


chess_board = [
    ["R", "N", "B", "Q", "K", "B", "N", "R"],
    ["P", "P", "P", "P", "P", "P", "P", "P"],
    ["", "", "", "", "", "", "", ""],
    ["", "", "", "", "", "", "", ""],
    ["", "", "", "", "", "", "", ""],
    ["", "", "", "", "", "", "", ""],
    ["p", "p", "p", "p", "p", "p", "p", "p"],
    ["r", "n", "b", "q", "k", "b", "n", "r"]
]




def draw_board():
    for row in range(ROWS):
        for col in range(COLS):
            color = WHITE if (row + col) % 2 == 1 else BLACK
            pygame.draw.rect(window, color, pygame.Rect(col * SQUARE_SIZE, row * SQUARE_SIZE, SQUARE_SIZE, SQUARE_SIZE))


def draw_pieces(chess_board):
    for row in range(ROWS):
        for col in range(COLS):
            piece = chess_board[row][col]
            if piece != "" :
                if piece.lower()!=piece:
                    piece_image = pygame.image.load(f"images/white/{piece}.png")
                    piece_image = pygame.transform.scale(piece_image,(90,90))# Replace with your chess piece images
                    window.blit(piece_image, pygame.Rect(col * SQUARE_SIZE, row * SQUARE_SIZE, SQUARE_SIZE, SQUARE_SIZE))
                else:
                    piece_image = pygame.image.load(f"images/black/{piece}.png")
                    piece_image = pygame.transform.scale(piece_image,(90,90))# Replace with your chess piece images
                    window.blit(piece_image, pygame.Rect(col * SQUARE_SIZE, row * SQUARE_SIZE, SQUARE_SIZE, SQUARE_SIZE))
                    


def main(chess_board):
    a = ""
    b = ""
    c = 'q'
    i = 0
    k=1
    user_side = selectSide()
    while True:
        if user_side == True:
            for event in pygame.event.get():
                if event.type == pygame.QUIT:
                    pygame.quit()
                    sys.exit()    
                if event.type == pygame.MOUSEBUTTONDOWN and i==1:
                    (x2,y2)=pygame.mouse.get_pos()
                    print((x2,y2))
                    b=get_position((x2,y2))
                    i=2
                if event.type == pygame.MOUSEBUTTONDOWN and i==0:
                    (x1,y1)=pygame.mouse.get_pos()
                    print((x1,y1))
                    i=1
                    a=get_position((x1,y1))
            if a and b and i==2:
                x = a+b
                print(x)
                legal_moves = list(board.legal_moves)
                try:
                    move = chess.Move.from_uci(x)
                    if (b == 'a1' or 'b1' or 'c1' or 'd1' or 'e1' or 'f1' or 'g1' or 'h1' or 'a8' or 'b8' or 'c8' or 'd8' or 'e8' or 'f8' or 'g8' or 'h8') and (chess.Move.from_uci(a + b + c) in legal_moves):
                        move = chess.Move.from_uci(a+b+c)
                        board.push(move)
                        print(board)
                        i=0
                        chess_board=board_to_numpy(board)
                        print(chess_board)
                        x = DCE.computerMove(board, 1)
                        chess_board=board_to_numpy(board)
                        print(chess_board)
                    
                    elif move in legal_moves:
                        board.push(move)
                        print(board)
                        i=0
                    
                        chess_board=board_to_numpy(board)
                        print(chess_board)
                        x = DCE.computerMove(board, 1)
                        chess_board=board_to_numpy(board)
                        print(chess_board)
                    
                    else:
                        raise ValueError("Invalid move")
                except ValueError as e:
                    print(e)
                    i = 0
                    a = ""
                    b = ""
                    continue
            window.fill(GRAY)
            draw_board()
            draw_pieces(chess_board)
            pygame.display.update()
            clock.tick(60)
        elif user_side == False:
            if k==1:
                chess_board=board_to_numpy(board)
                print(chess_board)
                x = DCE.computerMove(board, 1)
                chess_board=board_to_numpy(board)
                print(chess_board)
   
                k+=1
                for event in pygame.event.get():
                    if event.type == pygame.QUIT:
                        pygame.quit()
                        sys.exit()    
                    if event.type == pygame.MOUSEBUTTONDOWN and i==1:
                        (x2,y2)=pygame.mouse.get_pos()
                        print((x2,y2))
                        b=get_position((x2,y2))
                        i=2
                    if event.type == pygame.MOUSEBUTTONDOWN and i==0:
                        (x1,y1)=pygame.mouse.get_pos()
                        print((x1,y1))
                        i=1
                        a=get_position((x1,y1))
                if a and b and i==2:
                    x = a+b
                    print(x)
                    legal_moves = list(board.legal_moves)
                    try:
                        move = chess.Move.from_uci(x)
                        if (b == 'a1' or 'b1' or 'c1' or 'd1' or 'e1' or 'f1' or 'g1' or 'h1' or 'a8' or 'b8' or 'c8' or 'd8' or 'e8' or 'f8' or 'g8' or 'h8') and (chess.Move.from_uci(a + b + c) in legal_moves):
                            move = chess.Move.from_uci(a+b+c)
                            board.push(move)
                            print(board)
                            i=0
                            chess_board=board_to_numpy(board)
                            print(chess_board)
                            x = DCE.computerMove(board, 1)
                            chess_board=board_to_numpy(board)
                            print(chess_board)

                        elif move in legal_moves:
                            board.push(move)
                            print(board)
                            i=0
                   
                            chess_board=board_to_numpy(board)
                            print(chess_board)
                            x = DCE.computerMove(board, 1)
                            chess_board=board_to_numpy(board)
                            print(chess_board)
                       
                        else:
                            raise ValueError("Invalid move")
                    except ValueError as e:
                        print(e)
                        i = 0
                        a = ""
                        b = ""
                        continue
                window.fill(GRAY)
                draw_board()
                draw_pieces(chess_board)
                pygame.display.update()
                clock.tick(60)
            else:
                for event in pygame.event.get():
                    if event.type == pygame.QUIT:
                        pygame.quit()
                        sys.exit()    
                    if event.type == pygame.MOUSEBUTTONDOWN and i==1:
                        (x2,y2)=pygame.mouse.get_pos()
                        print((x2,y2))
                        b=get_position((x2,y2))
                        i=2
                    if event.type == pygame.MOUSEBUTTONDOWN and i==0:
                        (x1,y1)=pygame.mouse.get_pos()
                        print((x1,y1))
                        i=1
                        a=get_position((x1,y1))
                if a and b and i==2:
                    x = a+b
                    print(x)
                    legal_moves = list(board.legal_moves)
                    try:
                        move = chess.Move.from_uci(x)
                        if (b == 'a1' or 'b1' or 'c1' or 'd1' or 'e1' or 'f1' or 'g1' or 'h1' or 'a8' or 'b8' or 'c8' or 'd8' or 'e8' or 'f8' or 'g8' or 'h8') and (chess.Move.from_uci(a + b + c) in legal_moves):
                            move = chess.Move.from_uci(a+b+c)
                            board.push(move)
                            print(board)
                            i=0
                            chess_board=board_to_numpy(board)
                            print(chess_board)
                            x = DCE.computerMove(board, 1)
                            chess_board=board_to_numpy(board)
                            print(chess_board)
                        elif move in legal_moves:
                            board.push(move)
                            print(board)
                            i=0
                
                            chess_board=board_to_numpy(board)
                            print(chess_board)
                            x = DCE.computerMove(board, 1)
                            chess_board=board_to_numpy(board)
                            print(chess_board)
                    
                        else:
                            raise ValueError("Invalid move")
                    except ValueError as e:
                        print(e)
                        i = 0
                        a = ""
                        b = ""
                        continue
                window.fill(GRAY)
                draw_board()
                draw_pieces(chess_board)
                pygame.display.update()
                clock.tick(60)
if __name__ == "__main__":
    main(chess_board)
